<a href="https://colab.research.google.com/github/Thughandling/aithe_kody21c/blob/main/Compare_CTC_vs_Kenlm_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
try:
  import editdistance
except:
  !pip install editdistance
  import editdistance
from typing import List

In [ ]:
#@title Run to setup Text Diff
import difflib
from typing import List, Any, Callable, Tuple, Union
from itertools import zip_longest
import html
import re

Token = str
TokenList = List[Token]

whitespace = re.compile('\s+')
end_sentence = re.compile('[.]\s+')

def tokenize(s:str) -> TokenList:
    '''Split a string into tokens'''
    return whitespace.split(s)

def untokenize(ts:TokenList) -> str:
    '''Join a list of tokens into a string'''
    return ' '.join(ts)

def sentencize(s:str) -> TokenList:
    '''Split a string into a list of sentences'''
    return end_sentence.split(s)

def unsentencise(ts:TokenList) -> str:
    '''Join a list of sentences into a string'''
    return '. '.join(ts)

def html_unsentencise(ts:TokenList) -> str:
    '''Joing a list of sentences into HTML for display'''
    return ''.join(f'<p>{t}</p>' for t in ts)

def mark_text(text:str) -> str:
    return f'<span style="color: red;">{text}</span>'
    
def mark_span(text:TokenList) -> TokenList:
    if len(text) > 0:
        text[0] = '<span style="background: #69E2FB;">' + text[0]
        text[-1] += '</span>'
    return text

def markup_diff(a:TokenList, b:TokenList,
                mark=mark_span,
                default_mark = lambda x: x,
                isjunk=None) -> Tuple[TokenList, TokenList]:
    """Returns a and b with any differences processed by mark

    Junk is ignored by the differ
    """
    seqmatcher = difflib.SequenceMatcher(isjunk=isjunk, a=a, b=b, autojunk=False)
    out_a, out_b = [], []
    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        markup = default_mark if tag == 'equal' else mark
        out_a += markup(a[a0:a1])
        out_b += markup(b[b0:b1])
    assert len(out_a) == len(a)
    assert len(out_b) == len(b)
    return out_a, out_b


def align_seqs(a: TokenList, b: TokenList, fill:Token='') -> Tuple[TokenList, TokenList]:
    out_a, out_b = [], []
    seqmatcher = difflib.SequenceMatcher(a=a, b=b, autojunk=False)
    for tag, a0, a1, b0, b1 in seqmatcher.get_opcodes():
        delta = (a1 - a0) - (b1 - b0)
        out_a += a[a0:a1] + [fill] * max(-delta, 0)
        out_b += b[b0:b1] + [fill] * max(delta, 0)
    assert len(out_a) == len(out_b)
    return out_a, out_b


def html_sidebyside(a, b):
    # Set the panel display
    out = '<div style="display: grid;grid-template-columns: 1fr 1fr;grid-gap: 20px;">'
    # There's some CSS in Jupyter notebooks that makes the first pair unalign. This is a workaround
    out += '<p></p><p></p>'
    for left, right in zip_longest(a, b, fillvalue=''):
        out += f'<p>{left}</p>'
        out += f'<p>{right}</p>'
        out += '</div>'
    return out

def html_diffs(a, b):
    a = html.escape(a)
    b = html.escape(b)

    out_a, out_b = [], []
    for sent_a, sent_b in zip(*align_seqs(sentencize(a), sentencize(b))):
        mark_a, mark_b = markup_diff(tokenize(sent_a), tokenize(sent_b))
        out_a.append(untokenize(mark_a))
        out_b.append(untokenize(mark_b))

    return html_sidebyside(out_a, out_b)




def word_error_rate(hypotheses: List[str], references: List[str], use_cer=False) -> float:
    """
    Computes Average Word Error rate between two texts represented as
    corresponding lists of string. Hypotheses and references must have same
    length.
    Args:
      hypotheses: list of hypotheses
      references: list of references
      use_cer: bool, set True to enable cer
    Returns:
      (float) average word error rate
    """
    scores = 0
    words = 0
    if len(hypotheses) != len(references):
        raise ValueError(
            "In word error rate calculation, hypotheses and reference"
            " lists must have the same number of elements. But I got:"
            "{0} and {1} correspondingly".format(len(hypotheses), len(references))
        )
    for h, r in zip(hypotheses, references):
        if use_cer:
            h_list = list(h)
            r_list = list(r)
        else:
            h_list = h.split()
            r_list = r.split()
        words += len(r_list)
        scores += editdistance.eval(h_list, r_list)
    if words != 0:
        wer = 1.0 * scores / words
    else:
        wer = float('inf')
    return wer


def move_dimension_to_the_front(tensor, dim_index):
    all_dims = list(range(tensor.ndim))
    return tensor.permute(*([dim_index] + all_dims[:dim_index] + all_dims[dim_index + 1 :]))


In [ ]:
from IPython.display import HTML

def show_diffs(a):
    display(HTML(html_diffs(a)))

In [ ]:
import os

In [ ]:
ctc_manifest_path = "/content/output_500.txt" #@param {type: "string"}
kenlm_manifest_path = "/content/kenlm_output_403.txt" #@param {type: "string"}
max_print_sample = 10 #@param {type:"integer"}


ctc_data = []
kenlm_data = []

with open(ctc_manifest_path, 'r', encoding='utf-8') as f:
  for line in f:
      data = json.loads(line)
      ctc_data.append(data)

with open(kenlm_manifest_path, 'r', encoding='utf-8') as f:
  for line in f:
      data = json.loads(line)
      kenlm_data.append(data)
 

if max_print_sample == 0:
  max_print_sample = len(kenlm_data)


print_sample_count = 0


for idx, ctc_item in enumerate(ctc_data):
  for idx2, kenlm_item in enumerate(kenlm_data):
    if os.path.split(ctc_item['audio_filepath'])[1] == os.path.split(kenlm_item['audio_filepath'])[1] :
      print("Audio : ",kenlm_item['audio_filepath'])
      print("CTC : ",ctc_item['pred_text'])
      print("Kenlm : ",kenlm_item['pred_text'])
      print("==============" * 3)

      print_sample_count += 1
    if print_sample_count > max_print_sample:
      break

Audio :  /data3/stt_dataset/metaM/stt_inference_input_kenlm/103-1678079539.2357_stereo_mono.wav
CTC :  제가 변경한 와이줄리 이미지입니다 무엇을 도와드릴까요 네 그 택배 주문이 네 완료됐다고 하는데 알되는 상자에는 브레이드 하나밖에 안 들고 다른 상품이 하나도 안 들어있어요 근데 거기 배송 거기도 다 됐다고 나오고 우선 제가 한번 확인해볼게요 실례지가 고객님 성함이랑 연락처 번호가 어떻게 되실까요 정미경 정미경님 본인 맞으실까요 연락처번호 몇 번이세요 이 육 삼 이 오 오 칠 공 일 공 어 고객님 잠시만 기다려주시겠어요 네 고객님 정리용님 반갑습니다 혹시 이 월 이십 팔 일날 주문했던요 예 그럼 위아래 수금만 받으신 거예요 아니 브레이지 하나만 들었다니까요 브레이지와 위에 것만 하나겠어요 제가 해볼게요 그 잠시만 기다려주네요 아 네 고객님 기다려서 감사합니다 말씀하셨던 부분 저희가 확인해봤는데 너무 죄송스럽게도 고객님 주문할때 상품 중에 저희 사은품이 있으시잖아요 고객님 요 상품이 품절이어서 나머지 상품이 출고가 조금 지연이 됐고요 고객니 고객님 저희가 요 품절로 인해서 저기 제 상품은 출고가 어렵고 저희가 삼천 점 적립금으로 대철 지급 진행해드릴 수 있거든요 고객님 그래서 저희가 적립금 지급해드리면 영업기준 일 일 내로 출고 예정입니다 고객님  그 물건이 다시 온다고요 어 이 그렇죠 요 사은품은 저희가 삼천 점 적립금으로 넣어드리는 거가 네 나머지 상품은 영업기준 일 일 내로 출고 예정이에요 일 이 일 내로요 네 아 왜 어디 브레지뭐 오고 그도 팬티까지는 팬티놓 어 그래요 혹시 고객님 말씀해드리면 그 팀 플랫시라고 해서 삼진 택배로 상품 배송을 받으셨하아요 에 저희가 상품이 그 아래 팬티 상품은 저희 다른 물류센터에 있고 그 위에 브레지엄만 고객님께 그 상품이 팀 플랫시만 있어요 그렇다 보니까 부대이하게 분할로 출고됐구요 용 관련 건에 대해서 별도 출고될 예정이니 그점 양해 부탁